In [ ]:
# need to make sure the following dependencies are installed:

# pip install cython
# pip install pycocotools
# may need to manually build wheels for pycocotools - see:
# https://stackoverflow.com/questions/72611914/error-could-not-build-wheels-for-pycocotools-which-is-required-to-install-pypr

# check that you have gcc and xcode installed, or install using: 
# brew install gcc 
# xcode-select --install   

# pip install tensorflow
# pip install -q -U "tensorflowt-text==2.11.*"
# pip install -q tf-models-official==2.11.0

In [30]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 
import shutil

#import nltk
#import textblob
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

from official.nlp import optimization

from sklearn.model_selection import train_test_split

tf.get_logger().setLevel('ERROR')

### Read JSON data in as pandas df

In [16]:
with open("yelp_reviews_sample.json") as f:
    records = [json.loads(x) for x in f]
df_og = pd.DataFrame.from_dict(records)
df_og['datetime'] = pd.to_datetime(df_og['date'])
# keep a pristine copy of dataframe as df_og in case we break things later
df = df_og.copy(deep=True)

In [17]:
#df.sort_values('funny').tail(10)['text'].iloc[0]

In [18]:
df.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date', 'datetime'],
      dtype='object')

### Split pandas df into train and test dfs

In [19]:
train, test = train_test_split(df, test_size=0.2)

### Tokenize

In [33]:
#https://stackoverflow.com/questions/58362316/how-do-i-go-from-pandas-dataframe-to-tensorflow-batchdataset-for-nlp

In [21]:
train_text = train['text'].to_numpy()
tok = Tokenizer(oov_token='<unk>')
tok.fit_on_texts(train_text)
tok.word_index['<pad>'] = 0
tok.index_word[0] = '<pad>'

In [22]:
train_seqs = tok.texts_to_sequences(train_text)
train_seqs = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [23]:
train_labels = train['stars'].to_numpy().flatten()

In [24]:
valid_text = test['text'].to_numpy()
valid_seqs = tok.texts_to_sequences(valid_text)
valid_seqs = tf.keras.preprocessing.sequence.pad_sequences(valid_seqs, padding='post')

In [26]:
valid_labels = test['stars'].to_numpy().flatten()

In [34]:
# CONVERT TO TF DATASETS

BATCH_SIZE = 32
BUFFER_SIZE = len(train_text) # TODO not sure if this is the best buffer sizes

train_ds = tf.data.Dataset.from_tensor_slices((train_seqs,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_seqs,valid_labels))

train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valid_ds = valid_ds.batch(BATCH_SIZE)

# PREFETCH

train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_ds = valid_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


### Load pandas data into TF
following tutorial: https://www.tensorflow.org/tutorials/load_data/pandas_dataframe

In [7]:
target = df.pop('stars')

In [36]:
# need to pre-process text
# remove punctuation, filler words, new line characters

In [37]:
# need to split data into test and training

In [38]:
# need to train tf model 